# K means clustering  for search  

load data and clean 

In [1]:

from os import listdir
from os.path import isfile, join
from html.parser import HTMLParser
from bs4 import BeautifulSoup

def load_data():
    PATH = "../data/"
    contents = []
    files = [f"{PATH}{f}" for f in listdir(PATH) if isfile(join(PATH, f))]
    for file_path in files:
        with open(file_path, "r") as f:
            contents.append(f.read())
    return files, contents


def load_stop_words(path: str = "../stopwords.txt"):
    words = []
    with open(path, "r") as f:
        words = f.read().split("\n")
    assert len(words) != 0, "no stop words were found"
    return words


def clean_html(contents:list) -> list:
    result = []
    for html in contents:
        soup = BeautifulSoup(html, 'html.parser')
        result.append(soup.get_text())
    return result

def clean(contents: list, remove_stop_words=True):
    #contents = clean_html(contents)
    ascii_char = [chr(i) for i in range(0, 255)]
    numbers = "0123456789"
    non_acc_char = "\n,.()[]{}`/:-_*=\\<>|&%@?!\"'#" + numbers
    non_acc_tokens = ["https", "www", "com", "org", "license"]
    stop_words = [""] 
    if remove_stop_words: 
        stop_words = load_stop_words()
    for i, _ in enumerate(contents):
        for c in non_acc_char:
            contents[i] = contents[i].replace(c, " ")
        contents[i] = contents[i].split(" ")
        contents[i] = list(filter(lambda c: c != "", contents[i]))
        contents[i] = [t for t in contents[i] if not t in non_acc_tokens]
        contents[i] = [
            s.lower() for s in contents[i] if all(c in ascii_char for c in s)
        ]
        contents[i] = [t for t in contents[i] if not t in stop_words]
        for j, word in enumerate(contents[i]):
            if word[-1] == "s":
                contents[i][j] = word[:-1]
    return [" ".join(con) for con in contents]


def words_to_vec(words: str, labels: dict = {}):
    _words = words.split(" ")
    vec = []
    for word in _words:
        if word not in labels:
            labels[word] = len(labels)
        vec.append(labels[word])
    return vec, labels


### impl kmenas  

In [8]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import HashingVectorizer 

In [9]:
K = 4

In [13]:
files, content = load_data()
content = clean(content)
vectorizer = HashingVectorizer()
content = vectorizer.fit_transform(content)
model = KNeighborsClassifier(n_neighbors=K)
model.fit(content,files)

KNeighborsClassifier(n_neighbors=4)

In [11]:
def to_vec(q:str):
    global vectorizer
    q = clean([q])[0]
    return vectorizer.transform([q])

In [12]:
prob = model.predict_proba(to_vec("government agencies"))[0]
print(prob)
for i, p in enumerate(prob):
    if p > 0.05: 
        print(files[i]) 

[0.25 0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.
 0.25 0.  ]
../data/websites/QmYaMCtgVF46b5jTJ9n95F5yTgw9ZYCRNupBeJrvnphjTW
../data/websites/QmR4kqBs7sis2dBuhfriAd4szr7VPERqnEvqvgaYvMomSj
../data/websites/QmauhUbPerCeRjJAbPCoDAdASuzWJ9sMrx5k7FUPeSQDyU
../data/websites/Qma4TxpzdcYqDMzpgGAHYCyjrsbin2C6hTnXvjbMUFondJ


## Ball tree

In [3]:

from sklearn.neighbors import BallTree
import pandas as pd 

In [ ]:
files, content = load_data()
content = clean(content)
df = pd.DataFrame()